In [1]:
!pip install bs4

     |████████████████████████████████| 122kB 13.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

request data from the webpage and create a soup object using the data

In [3]:
html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(html).text
soup = BeautifulSoup(html_data,'html5lib')

process each cell in the table and store as a dictionary and the append the dictionary to the table_contents list. Ignore cells with a borough that is Not assigned

In [4]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        row_left = row.span.text.split('(')
        cell['Borough'] = row_left[0]
        cell['Neighborhood'] = row_left[1].strip(')').replace('/',',').replace(')',' ').strip(' ')
        table_contents.append(cell)

Create a dataframe using the table_contents list

In [5]:
df = pd.DataFrame(table_contents)

In [6]:
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto Stn A',
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                       'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEastToronto':'East York/East Toronto',
                                       'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [7]:
df.shape

(103, 3)

In [8]:
geo_data = pd.read_csv(r'Geospatial_Coordinates.csv')

In [9]:
geo_data.shape

(103, 3)

In [10]:
geo_data = geo_data.rename(columns={'Postal Code':'PostalCode'})

In [11]:
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#new_data = pd.merge(df,geo_data[['Postal Code','Latitude','Longitude']],on = 'PostalCode')
new_data = pd.merge(df,geo_data)

In [13]:
new_data.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Part 3

Select data where Borough contains "Toronto"

In [14]:
data = new_data[new_data['Borough'].str.contains('Toronto')]
data = data.reset_index(drop=True)

In [15]:
data[['Borough','Neighborhood','Latitude','Longitude']].head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,East Toronto,The Beaches,43.676357,-79.293031
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [37]:
len(data['Neighborhood'].unique())

39

check the dataset size

In [38]:
print('The dataframe has {} bouroughs'.format(
        len(data['Borough'].unique())))

The dataframe has 7 bouroughs


Use geopy library to get the latitude and longitude values of Toronto

In [16]:
!pip install geopy

In [17]:
from geopy.geocoders import Nominatim

In [18]:
address = 'Toronto'
geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torondo are {},{}.'.format(latitude,longitude))

The geograpical coordinate of Torondo are 43.6534817,-79.3839347.


Create a map of Toronto with neighborhoods superimposed on top

In [19]:
!pip install folium

In [20]:
import folium

In [21]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)
# add markers to map
for lat,lng,borough,neighborhood in zip(data['Latitude'],data['Longitude'],data['Borough'],data['Neighborhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
                        [lat,lng],radius=5,popup=label,color='blue',
                        fill=True,fill_color='#3186cc',fill_opacity=0.7,
                        parse_html=False).add_to(map_toronto)
map_toronto

Utilizing the Foursquare API to explore the neighborhoods and segment them

In [22]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'CFHOO33ZUAPC2CJMCHP50EVPXFLT4UHM1TMMXGWNNOP3ZVPV' # your Foursquare ID

CLIENT_SECRET = '33BJYOZHMPVNCUKG2Q1LNJYZR0BC0J3P2PEMXLSRWXNMJHIM' # your Foursquare Secret
VERSION = '20210326'
#VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CFHOO33ZUAPC2CJMCHP50EVPXFLT4UHM1TMMXGWNNOP3ZVPV
CLIENT_SECRET:33BJYOZHMPVNCUKG2Q1LNJYZR0BC0J3P2PEMXLSRWXNMJHIM


Let's explore the first neighborhood in our dataframe

In [23]:
data.loc[0,'Neighborhood']
neighborhood_latitude = data.loc[0,'Latitude']
neighborhood_longitude = data.loc[0,'Longitude']
neighborhood_name = data.loc[0,'Neighborhood']
print('Latitude and longitude values of {} are {},{}.'.format(neighborhood_name,
                                                              neighborhood_latitude,
                                                              neighborhood_longitude))

Latitude and longitude values of Regent Park , Harbourfront are 43.6542599,-79.3606359.


Now, let's get the top 100 venues that are in Regent Part, Harbourfront within a radius of 500 meters

In [24]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,neighborhood_longitude,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CFHOO33ZUAPC2CJMCHP50EVPXFLT4UHM1TMMXGWNNOP3ZVPV&client_secret=33BJYOZHMPVNCUKG2Q1LNJYZR0BC0J3P2PEMXLSRWXNMJHIM&v=20210326&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [25]:
results = requests.get(url).json()

Function that extracts the category of the venue

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure into a pandas dataframe

In [27]:
from pandas.io.json import json_normalize

In [28]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type,axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

In [32]:
print(toronto_venues.shape)
toronto_venues.shape

(1580, 7)


(1580, 7)

In [33]:
toronto_venues.groupby('Neighborhood').count().shape

(39, 6)

In [58]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


## Analyze Each Neighborhood

In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_onehot.shape

(1580, 232)

Next, let's group rows by neighborhood and by takin gthe mean of the frequency of occurrence of each category

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000,0.0,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.040000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,Central Bay Street,0.015152,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015152,0.0,0.0,0.015152,0.0,0.0
4,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [37]:
toronto_grouped.shape

(39, 232)

In [42]:
selected = toronto_grouped[(toronto_grouped['Chinese Restaurant']!=0) & (toronto_grouped['Hotel']!=0) & (toronto_grouped['Park']!=0)]
selected[['Neighborhood','Chinese Restaurant','Hotel','Park']]

,Neighborhood,Chinese Restaurant,Hotel,Park
14,"Garden District, Ryerson",0.01,0.02,0.01
15,"Harbourfront East , Union Station , Toronto Is...",0.01,0.04,0.02


Let's print each neighborhood along with the top 5 most common venues

In [63]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood']==hood].T.reset_index()
    
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Pharmacy  0.03
3   Cheese Shop  0.03
4        Bakery  0.03


----Brockton , Parkdale Village , Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3              Bakery  0.05
4  Italian Restaurant  0.05


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                 venue  freq
0       Airport Lounge  0.13
1      Airport Service  0.13
2     Airport Terminal  0.13
3                Plane  0.07
4  Rental Car Location  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1      Sandwich Place  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Baby Store  

Now Let's create the new dataframe and display the top 10 venues for each neighborhood

In [64]:
import numpy as np

In [65]:
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10
indicators = ['st','nd','rd']
columns= ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Pharmacy,Cheese Shop,Beer Bar,Japanese Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
2,"CN Tower , King and Spadina , Railway Lands , ...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Portuguese Restaurant,Bookstore,Japanese Restaurant
4,Christie,Grocery Store,Café,Park,Nightclub,Candy Store,Athletics & Sports,Italian Restaurant,Restaurant,Baby Store,Coffee Shop
5,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Yoga Studio,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gay Bar
6,"Commerce Court , Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Gym,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Deli / Bodega
7,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Café,Gym,Sushi Restaurant,Brewery,Seafood Restaurant
8,Davisville North,Gym,Hotel,Pizza Place,Playground,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Garden Center
9,"Dufferin , Dovercourt Village",Pharmacy,Bakery,Grocery Store,Furniture / Home Store,Middle Eastern Restaurant,Music Venue,Park,Café,Brewery,Bar


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [67]:
from sklearn.cluster import KMeans

In [69]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 4,
       0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [70]:
#neighborhoods_venues_sorted.drop(columns=['Cluster Labels'])
neighborhoods_venues_sorted.shape

(39, 11)

In [71]:
neighborhoods_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)

In [72]:
toronto_merged = data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
                        
toronto_merged.shape

(39, 16)

In [73]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Theater,Yoga Studio,Dessert Shop,Shoe Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Hotel,Pizza Place,Burger Joint
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Beer Bar,Gastropub,Cosmetics Shop,Cocktail Bar,Gym,Farmers Market,Hotel,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Health Food Store,Trail,Pub,Wings Joint,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Pharmacy,Cheese Shop,Beer Bar,Japanese Restaurant


Finally, let's visualize the resulting clusters

In [74]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [75]:
# create map
map_clusters = folium.Map(location=[latitude,longitude],zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
    
       
map_clusters

Examine Cluster

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,0,Park,Health Food Store,Trail,Pub,Wine Bar,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
9,East YorkEast Toronto,0,Park,Convenience Store,Metro Station,Wine Bar,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
18,Central Toronto,0,Park,Bus Line,Swim School,Wine Bar,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Yoga Studio,Dessert Shop
1,Downtown Toronto,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Cosmetics Shop,Lingerie Store,Pizza Place,Diner
2,Downtown Toronto,1,Café,Coffee Shop,Gastropub,Cosmetics Shop,Cocktail Bar,Department Store,Moroccan Restaurant,Lingerie Store,Italian Restaurant,Farmers Market
4,Downtown Toronto,1,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Restaurant,Cheese Shop,Jazz Club
5,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop,Thai Restaurant
6,Downtown Toronto,1,Grocery Store,Café,Coffee Shop,Park,Bank,Candy Store,Italian Restaurant,Baby Store,Restaurant,Nightclub
7,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Deli / Bodega,Clothing Store,Thai Restaurant,Hotel,Gym,Salad Place,Sushi Restaurant
8,West Toronto,1,Pharmacy,Bakery,Music Venue,Brewery,Café,Bar,Bank,Supermarket,Middle Eastern Restaurant,Brazilian Restaurant
10,Downtown Toronto,1,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Italian Restaurant,Fried Chicken Joint,Restaurant,Brewery,Sporting Goods Shop
11,West Toronto,1,Bar,Café,Vegetarian / Vegan Restaurant,Asian Restaurant,Restaurant,Vietnamese Restaurant,Men's Store,Coffee Shop,Korean Restaurant,Italian Restaurant


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Gym,Playground,Lawyer,Tennis Court,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,3,Garden,Home Service,Wine Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,4,Park,Trail,Jewelry Store,Sushi Restaurant,Wine Bar,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
33,Downtown Toronto,4,Park,Playground,Trail,Wine Bar,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
